In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


<center style="font-size: 25px; color:red">Imports</center>

In [2]:
### EDA
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
# plotly.express: 그래프 그리는 plotly 라이브러리, 완성형 보편적인 그래프 그릴 수 있음 (easy)
import matplotlib.pyplot as plt
import plotly.graph_objects as go
# plotly.graph_objects: 그래프 그리는 plotly 라이브러리, 커스터마이징한 그래프 그릴 수 있음 (expert)
from plotly.subplots import make_subplots

### pre-processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
# NaN 채워주는 라이브러리

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<center style="font-size: 25px; color:red">Data Loading and Preparation</center>

<span style="font-size: 18px; color:blue">Data Loading and Preparation</span>

In [3]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12
STRATEGY = 'median'

In [4]:
### train data
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train.shape

(8693, 14)

In [6]:
train.count()

PassengerId     8693
HomePlanet      8492
CryoSleep       8476
Cabin           8494
Destination     8511
Age             8514
VIP             8490
RoomService     8512
FoodCourt       8510
ShoppingMall    8485
Spa             8510
VRDeck          8505
Name            8493
Transported     8693
dtype: int64

In [7]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [8]:
print(f'Number of rows in train data: {train.shape[0]}')
print(f'Number of columns in train data: {train.shape[1]}')
print(f'Number of values in train data: {train.count().sum()}')
print(f'Number of missing values in train data: {sum(train.isna().sum())}')

Number of rows in train data: 8693
Number of columns in train data: 14
Number of values in train data: 119378
Number of missing values in train data: 2324


<span style="font-size: 18px; color:blue">Column Wise missing values</span>

In [9]:
print(train.isna().sum().sort_values(ascending = False))

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64


<span style="font-size: 18px; color:blue">Basic statistics of training data</span>

In [10]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


<span style="font-size: 18px; color:blue">Exploring Test Data</span>

<span style="font-size: 18px; color:blue">Quick View of Test Data</span>

In [11]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [12]:
print(f'Number of rows in test data: {test.shape[0]}')
print(f'Number of columns in test data: {test.shape[1]}')
print(f'Number of values in test data: {test.count().sum()}')
print(f'Number of missing values in test data: {sum(test.isna().sum())}')

Number of rows in test data: 4277
Number of columns in test data: 13
Number of values in test data: 54484
Number of missing values in test data: 1117


<span style="font-size: 18px; color:blue">Column Wise missing values</span>

In [13]:
print(test.isna().sum().sort_values(ascending = False))

FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64


<span style="font-size: 18px; color:blue">Basic statistics of training data</span>

In [14]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


<span style="font-size: 18px; color:blue">Submission File</span>

<span style="font-size: 18px; color:blue">Quick view of Submission File</span>

In [15]:
submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


<center style="font-size: 25px; color:red">EDA</center>

<span style="font-size: 18px; color:blue">Overview of Data</span>

In [16]:
train.drop(columns = ["PassengerId"], inplace = True)
test.drop(columns = ["PassengerId"], inplace = True)
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col != TARGET]
RANDOM_STATE = 12

In [17]:
print(FEATURES)

['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name']


In [18]:
train

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [19]:
train.iloc[:,:-1].describe().T.sort_values(by = 'std', ascending = False)

,count,mean,std,min,25%,50%,75%,max
FoodCourt,8510.0,458.077203,1611.489240,0.0,0.0,0.0,76.0,29813.0
VRDeck,8505.0,304.854791,1145.717189,0.0,0.0,0.0,46.0,24133.0
Spa,8510.0,311.138778,1136.705535,0.0,0.0,0.0,59.0,22408.0
RoomService,8512.0,224.687617,666.717663,0.0,0.0,0.0,47.0,14327.0
ShoppingMall,8485.0,173.729169,604.696458,0.0,0.0,0.0,27.0,23492.0
Age,8514.0,28.827930,14.489021,0.0,19.0,27.0,38.0,79.0


<span style="font-size: 18px; color:blue">Null Value Distribution</span>

<span style="font-size: 18px; color:blue">Column wise Null Distribution</span>

In [20]:
test.isna().sum()

HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [21]:
test_null = pd.DataFrame(test.isna().sum())

In [22]:
test_null

,0
HomePlanet,87
CryoSleep,93
Cabin,100
Destination,92
Age,91
VIP,93
RoomService,82
FoodCourt,106
ShoppingMall,98
Spa,101


In [23]:
test_null = test_null.sort_values(by = 0, ascending = False)

In [24]:
test_null

,0
FoodCourt,106
Spa,101
Cabin,100
ShoppingMall,98
Name,94
CryoSleep,93
VIP,93
Destination,92
Age,91
HomePlanet,87


In [25]:
train_null = pd.DataFrame(test.isna().sum())
train_null = train_null.sort_values(by = 0, ascending = False)[:-1]

In [26]:
fig = make_subplots(rows = 1,
                    cols = 2,
                    column_titles = ["Train Data","Test Data"],
                    x_title = "Missing Values")
fig.show()

In [27]:
train_null[0]

FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
Name: 0, dtype: int64

In [28]:
train_null.index

Index(['FoodCourt', 'Spa', 'Cabin', 'ShoppingMall', 'Name', 'CryoSleep', 'VIP',
       'Destination', 'Age', 'HomePlanet', 'RoomService'],
      dtype='object')

In [29]:
fig.add_trace(go.Bar(x=train_null[0],
                     y=train_null.index,
                     orientation='h'),
                     1,1)
# orientation='h': horizontal Bar Chart 
fig.show()

In [30]:
fig.add_trace(go.Bar(x=test_null[0],
                     y=test_null.index,
                     orientation='h'),
                     1,2)
fig.show()

In [31]:
fig.update_layout(showlegend=False, title_text='Column wise Null Value Distribution',title_x=0.5)

<span style="font-size: 18px; color:blue">Row wise Null Distribution</span>

In [32]:
missing_train_row = train.isna().sum(axis = 1)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()

In [33]:
missing_train_row

,index,0
0,0,0.759922
1,1,0.214771
2,2,0.023352
3,3,0.001956


In [34]:
train.isna().sum(axis = 1).unique()

array([0, 1, 2, 3])

In [35]:
train.shape[0]

8693

In [36]:
train.isna().sum(axis = 1).value_counts()

0    6606
1    1867
2     203
3      17
dtype: int64

In [37]:
train.isna().sum(axis=1)

0       0
1       0
2       0
3       0
4       0
       ..
8688    0
8689    0
8690    0
8691    0
8692    0
Length: 8693, dtype: int64

In [38]:
missing_train_row

,index,0
0,0,0.759922
1,1,0.214771
2,2,0.023352
3,3,0.001956


In [39]:
missing_train_row.value_counts()

index  0       
0      0.759922    1
1      0.214771    1
2      0.023352    1
3      0.001956    1
dtype: int64

In [40]:
missing_test_row = test.isna().sum(axis = 1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()

In [41]:
missing_test_row

,index,0
0,0,0.767126
1,1,0.205518
2,2,0.026420
3,3,0.000935


In [42]:
missing_train_row.columns = ['no','count']
missing_test_row.columns = ['no','count']
missing_train_row["count"] = missing_train_row["count"]*100
missing_test_row["count"] = missing_test_row["count"]*100

In [43]:
missing_train_row

,no,count
0,0,75.992178
1,1,21.477051
2,2,2.335212
3,3,0.195560


In [44]:
fig = make_subplots(rows = 1,
                    cols = 2,
                    column_titles = ["Train Data","Test Data"],
                    x_title = "Missing Values")
fig.add_trace(go.Bar(x=missing_train_row["no"],
                     y=missing_train_row["count"]),
                     1,1)
fig.add_trace(go.Bar(x=missing_test_row["no"],
                     y=missing_test_row["count"]),
                     1,2)
fig.update_layout(showlegend=False, title_text='Column wise Null Value Distribution',title_x=0.5)

<span style="font-size: 18px; color:blue">Continous and Categorical Data Distribution</span>

In [45]:
train.dtypes

HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [46]:
df = pd.concat([train[FEATURES], test[FEATURES]], axis = 0)
text_features = ["Cabin","Name"]
cat_features = [col for col in FEATURES if df[col].nunique()<25 and col not in text_features]
cont_features = [col for col in FEATURES if df[col].nunique()>=25 and col not in text_features]

del df

print(len(FEATURES))
print(len(cat_features))
print(len(cont_features))
print(len(text_features))

12
4
6
2


In [47]:
labels = ['categorical','continuous', 'text']
values = [len(cat_features),len(cont_features),len(text_features)]

fig = go.Figure(data = [go.Pie(
                            labels = labels,
                            values = values)])
fig.show()

<span style="font-size: 18px; color:blue">Imputing Missing values</span>

In [48]:
imputer_cols = ["Age", "FoodCourt","ShoppingMall","Spa","VRDeck","RoomService"]
imputer = SimpleImputer(strategy = STRATEGY)
imputer.fit(train[imputer_cols])

train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])

train["HomePlanet"].fillna('Z', inplace =True)
test["HomePlanet"].fillna('Z', inplace =True)

<span style="font-size: 18px; color:blue">Encoding Categorical Features</span>

In [49]:
label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train ,test = label_encoder(train,test ,label_cols)


<center style="font-size: 25px; color:red">Modeling</center>

<span style="font-size: 18px; color:blue">27 Different Classifiers(LAZY PREDICT)</span>

In [50]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

In [51]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score


import time
import warnings
warnings.filterwarnings('ignore')

학습과 테스트, feature와 target 데이터 분리

In [52]:
train.drop(["Name","Cabin"],axis=1,inplace=True)
test.drop(["Name","Cabin"],axis=1,inplace=True)
X = train.drop(TARGET,axis=1)
y = train[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=12,
                                                    test_size=0.33)

### LazyPredict란?
- 오픈소스 머신러닝 자동화 관련 파이썬 오픈소스 프로젝트
- 코드 한 줄로 여러 ML 모델을 불러와 학습시키고 결과를 확인할 수 있다
- 여러 모델들의 성능 지표도 비교할 수 있어 성능이 더 좋은 모델을 찾을 수 있다
- 파라미터를 조정은 따로 불가하다

In [53]:
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=False,
                     random_state=12,
                     classifiers='all')

models, predictions = clf.fit(X_train, X_test, y_train, y_test)
clear_output()

<span style="font-size: 18px; color:blue">TOP 15 Models</span>

In [54]:
models[:15]

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.79,0.79,0.79,0.79,0.88
XGBClassifier,0.78,0.78,0.78,0.78,0.33
AdaBoostClassifier,0.77,0.77,0.77,0.77,0.30
RandomForestClassifier,0.77,0.77,0.77,0.77,0.84
NuSVC,0.77,0.77,0.77,0.77,1.32
LogisticRegression,0.77,0.77,0.77,0.77,0.03
SVC,0.77,0.77,0.77,0.77,1.12
ExtraTreesClassifier,0.76,0.76,0.76,0.76,0.69
LinearSVC,0.76,0.76,0.76,0.76,0.44


<span style="font-size: 18px; color:blue">Visualizing Results</span>

In [55]:
line = px.line(data_frame = models, y=["Accuracy"], markers = True)
line.update_xaxes(title="Model",
                  rangeslider_visible=False)
line.update_yaxes(title="Accuracy")
line.update_traces(line_color="red")
line.update_layout(showlegend=True,
                  title={
                      'text':'Accuracy vs Model',
                      'y':0.94,
                      'x':0.5,
                      'xanchor':'center',
                      'yanchor':'top'
                  })
line.show()

LGBMClassifier

In [56]:
line = px.line(data_frame=models,y=["ROC AUC", "F1 Score"],markers=True)
line.update_xaxes(title="Model",
                  rangeslider_visible=False)
line.update_yaxes(title="ROC AUC SCORE")
line.update_layout(showlegend=True,
                title = {
                    'text':'ROC AUC and F1 Score vs Model',
                    'y':0.94,
                    'x':0.5,
                    'xanchor':'center',
                    'yanchor':'top'})

line.show()

In [57]:
line = px.line(data_frame=models, y=["Time Taken"],markers=True)
line.update_xaxes(title='Model',
                 rangeslider_visible=False)
line.update_yaxes(title="Time(s)")
line.update_traces(line_color="purple")
line.update_layout(showlegend=True,
                  title = {
                      'y':0.94,
                      'x':0.5,
                      'xanchor':'center',
                      'yanchor':'top'})
line.show()

<span style="font-size: 18px; color:blue">LGBM Classifier(5 FOLDS)</span>

In [58]:
lgb_params = {
    'objective': 'binary',
    'n_estimators':50,
    'learning_rate':0.08
}

lgb_predictions = 0
FOLDS = 5
lgb_scores = []
lgb_fimp = []
LGBM_FEATURES = list(train.columns)[:-1]
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
for fold, (train_idx,valid_idx) in enumerate(skf.split(train[LGBM_FEATURES],train[TARGET])):
    print(10*"=",f"Fold={fold+1}",10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][LGBM_FEATURES],train.iloc[valid_idx][LGBM_FEATURES]
    y_train, y_valid = train[TARGET].iloc[train_idx], train[TARGET].iloc[valid_idx]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train, verbose=0)
    
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(y_valid,preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    
    fim = pd.DataFrame(index = LGBM_FEATURES,
                      data=model.feature_importances_,
                      columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    print(f"Fold={fold+1},Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
    test_preds = model.predict(test[LGBM_FEATURES])
    lgb_predictions += test_preds/FOLDS
print("")
print("Mean Accuracy: ", np.mean(lgb_scores))

========== Fold=1 ==========
Fold=1,Accuracy score: 0.81%, Run Time: 0.46s
========== Fold=2 ==========
Fold=2,Accuracy score: 0.78%, Run Time: 0.42s
========== Fold=3 ==========
Fold=3,Accuracy score: 0.79%, Run Time: 0.43s
========== Fold=4 ==========
Fold=4,Accuracy score: 0.79%, Run Time: 0.44s
========== Fold=5 ==========
Fold=5,Accuracy score: 0.81%, Run Time: 0.42s

Mean Accuracy:  0.7958136330880743


<span style="font-size: 18px; color:blue">Feature Importance</span>

In [59]:
lgbm_fis_df = pd.concat(lgb_fimp, axis=1).head(15)
lgbm_fis_df.sort_values('1_importance').plot(kind='barh', figsize=(15, 10),
                                       title='Feature Importance Across Folds')
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_34/1316425447.py", line 2, in <module>
    lgbm_fis_df.sort_values('1_importance').plot(kind='barh', figsize=(15, 10),
  File "/opt/conda/lib/python3.10/site-packages/pandas/plotting/_core.py", line 920, in __call__
    return plot_backend.plot(data, x=x, y=y, kind=kind, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/pandas/plotting/_core.py", line 1886, in _get_plot_backend
  File "/opt/conda/lib/python3.10/site-packages/pandas/plotting/_core.py", line 1817, in _load_backend
    # Fall back to unregistered, module name approach.
ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3

In [60]:
submission[TARGET] = lgb_predictions.astype('bool')
submission.to_csv("submission.csv",index=False)
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
